<a href="https://colab.research.google.com/github/ra-MANUJ-an/MIDAS-PROJECT/blob/main/Project_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTS**

Import necessary packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from zipfile import ZipFile # extracting zip files
import random # shuffling images
import pickle
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import h5py
import math
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)
import gzip

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


**RESNET UTILITIES**

taken from : https://github.com/priya-dwivedi/Deep-Learning/blob/master/resnet_keras/resnets_utils.py

In [ ]:
#resnets_utils.py

def load_dataset(): 
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels
    
    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels
    
    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]
    
    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    return mini_batches

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y
    
def forward_propagation_for_predict(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters
    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3'] 
                                                           # Numpy Equivalents:
    Z1 = tf.add(tf.matmul(W1, X), b1)                      # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                    # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                     # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                    # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)                     # Z3 = np.dot(W3,Z2) + b3
    
    return Z3
    
def predict(X, parameters):
    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])
    
    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}
    
    x = tf.placeholder("float", [12288, 1])
    
    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})
         
    return prediction 

Extracting Files from "trainPart1.zip"

In [ ]:
file_name = "trainPart1.zip"

with ZipFile(file_name, 'r') as zip :
  zip.extractall()
  print('Done')

Done


In [ ]:
DATADIRECTORY = "/content/train"
CATEGORIES = ["Sample001", "Sample002", "Sample003", "Sample004", "Sample005", "Sample006", "Sample007", "Sample008", "Sample009", "Sample010",
 "Sample011", "Sample012", "Sample013", "Sample014", "Sample015", "Sample016", "Sample017", "Sample018", "Sample019", "Sample020",
 "Sample021", "Sample022", "Sample023", "Sample024", "Sample025", "Sample036", "Sample027", "Sample028", "Sample029", "Sample030",
 "Sample031", "Sample032", "Sample033", "Sample034", "Sample035", "Sample036", "Sample037", "Sample038", "Sample039", "Sample040",
 "Sample041", "Sample042", "Sample043", "Sample044", "Sample045", "Sample046", "Sample047", "Sample048", "Sample049", "Sample050",
 "Sample051", "Sample052", "Sample053", "Sample054", "Sample055", "Sample056", "Sample057", "Sample058", "Sample059", "Sample060",
 "Sample061", "Sample062"]

In [ ]:
classSize = len(CATEGORIES)
print(classSize)

In [ ]:
IMG_SIZE = 100

In [ ]:
trainingData = []

def createTrainingData():
    for category in CATEGORIES:

        path = os.path.join(DATADIRECTORY,category)
        classNum = CATEGORIES.index(category)  # get the classification

        for img in tqdm(os.listdir(path)):
            try:
                imgArray = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                newArray = cv2.resize(imgArray, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                trainingData.append([newArray, classNum])  # add this to our trainingData
            except Exception as e:  # keeps output cleans
                pass

createTrainingData()
print(len(trainingData))

random.shuffle(trainingData)
# we are shuffling the data so that classifier does not 
# look at same type of images for a long time

X = []
y = []

for features, label in trainingData :
  X.append(features)
  y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)


100%|██████████| 40/40 [00:00<00:00, 87.03it/s]

2480


**SMALL RESIDUAL NETWORK**

Did not work very well

In [ ]:
# Normalize image vectors
X = X/255.0

model = Sequential()
# conv layer
model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # converts our 3D feature vectors to 1D feature vectors

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(classSize))  # our output layer. 
# Softmax for probability distribution
model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # what to track

In [ ]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)
model.fit(X, y, batch_size=32, epochs=20)

Epoch 1/20
78/78 [==============================] - 12s 125ms/step - loss: 4.2050 - accuracy: 0.0093
Epoch 2/20
78/78 [==============================] - 9s 112ms/step - loss: 4.0768 - accuracy: 0.0291
Epoch 3/20
78/78 [==============================] - 9s 111ms/step - loss: 3.8147 - accuracy: 0.0709
Epoch 4/20
78/78 [==============================] - 9s 111ms/step - loss: 3.3075 - accuracy: 0.1504
Epoch 5/20
78/78 [==============================] - 9s 111ms/step - loss: 2.9634 - accuracy: 0.2087
Epoch 6/20
78/78 [==============================] - 9s 111ms/step - loss: 2.4837 - accuracy: 0.3041
Epoch 7/20
78/78 [==============================] - 9s 112ms/step - loss: 2.1661 - accuracy: 0.3804
Epoch 8/20
78/78 [==============================] - 9s 111ms/step - loss: 1.9116 - accuracy: 0.4351
Epoch 9/20
78/78 [==============================] - 9s 111ms/step - loss: 1.7307 - accuracy: 0.4797
Epoch 10/20
78/78 [==============================] - 9s 112ms/step - loss: 1.4904 - accuracy: 0.529

**RESNET50**

Implementation inspiration from
- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet's GitHub repository: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py

In [ ]:
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value 
    X_shortcut = X
    
    # Skips over 3 hidden layers

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
        
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Add shortcut value to main path
    X = Add()([X,X_shortcut])
    # pass it through a RELU activation
    X = Activation('relu')(X)
    
    return X

In [ ]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Extract Filters
    F1, F2, F3 = filters
    
    # Save the input value as shortcut
    X_shortcut = X

    # MAIN PATH #

    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(F2, (f, f), strides = (1,1), name = conv_name_base + '2b', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # SHORTCUT PATH #
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Add shortcut value to main path
    X = Add()([X, X_shortcut])
    # pass through RELU activation
    X = Activation('relu')(X)
    
    return X

In [ ]:
# GRADED FUNCTION: ResNet50
# returns resnet model
def ResNet50(input_shape, classes):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # Zero-Padding, pads the input with pad(3,3)
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D((2, 2), padding='same')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape = (IMG_SIZE, IMG_SIZE, 1), classes = classSize)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Normalize image vectors
X = X/255.

# Convert training and test labels to one hot matrices
y = convert_to_one_hot(y, classSize).T

print ("number of training examples = " + str(X.shape[0]))
print ("X shape: " + str(X.shape))
print ("y shape: " + str(y.shape))

number of training examples = 2480
X shape: (2480, 100, 100, 1)
y shape: (2480, 62)


In [ ]:
# model.fit(X, y, epochs = 6, batch_size = 32)
model.fit(X, y, batch_size=32, epochs=20)

Epoch 1/20
78/78 [==============================] - 25s 231ms/step - loss: 11.2854 - accuracy: 0.0259
Epoch 2/20
78/78 [==============================] - 17s 218ms/step - loss: 5.1499 - accuracy: 0.0564
Epoch 3/20
78/78 [==============================] - 17s 219ms/step - loss: 3.6131 - accuracy: 0.1411
Epoch 4/20
78/78 [==============================] - 17s 219ms/step - loss: 3.0828 - accuracy: 0.2512
Epoch 5/20
78/78 [==============================] - 17s 219ms/step - loss: 2.4039 - accuracy: 0.3451
Epoch 6/20
78/78 [==============================] - 17s 219ms/step - loss: 1.9473 - accuracy: 0.4875
Epoch 7/20
78/78 [==============================] - 17s 219ms/step - loss: 1.4202 - accuracy: 0.5808
Epoch 8/20
78/78 [==============================] - 17s 219ms/step - loss: 1.1359 - accuracy: 0.6709
Epoch 9/20
78/78 [==============================] - 17s 219ms/step - loss: 0.9673 - accuracy: 0.7104
Epoch 10/20
78/78 [==============================] - 17s 219ms/step - loss: 0.9160 - accur

Resnet model did not performed well compared to sequential model when image size was ~ (30 X 30) but outperformed it when it was increased to ~(100 X 100)